# Energy Reconstruction Using CNN

In [4]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = os.getcwd()+'\\simdata'

## Data Input

In [2]:
x, y = load_preprocessed(simPrefix, 'train')

ZeroDivisionError: division by zero

In [ ]:
print(x.shape)
print(y.keys())
# each station has 2 tanks, each tank has 2 DOMs (high/log gain)
# each tank measures charge and time
# each station gives 2 charges and 2 times, 4 pieces of data
# x: (num, i, j, data)
# charge measured in VEM, vertical equivalent muon

# 'dir' is true direction, rest of dir are reconstructed by simulations
# plane assumes shower is flat plane
# laputop performs likelihood analysis
# small compromises between

In [ ]:
# 85/15 split for training/testing
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

## Model Training

### Alpha Model
- Input: no charge merge, no time layers included, normalized data
- Layers: Two convolutional
- Output: Energy

In [ ]:
# Name for model
key = 'first_model'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':None, 't':False, 'normed':True}

In [ ]:
# Create model
model = Sequential(name=nameModel(prep, 'test'))  # Automatic naming for flexible assessment later
# Add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(1)) # No activation function for last layer of regression model

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [ ]:
# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)
temp_y = energy

In [ ]:
# Train
history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=3)

In [ ]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
model.save('model_%s.h5' % key)
model.evaluate(x_i[testCut], temp_y[testCut])

## Your task

- **Create your own model**
- Replace the model here w/ *simplified* form of Brandon's model (focus: including zenith)
- change the zenith input to cosine(zenith) input

In [ ]:
history.history